# Using Linear Regression To Predict Total Storage Of An Aquifer

Linear regression is used here becase we need non categorical values and since SVM and most other machine learning methods are basically just used for catrgoring. 

In [109]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale
from sklearn.svm import SVC, SVR
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

Import the geophysical data

In [110]:
df = pd.read_csv("paper2 VES Table Final.csv", header=0, sep=",")

In [111]:
df.head()

,VES S/N,Easting (Min),Northing (Min),Elev(m),No. of Layer,Curve Types,Res. of layer\np1/p2/�pn-1,Thickness of layers h1/h2/h3/h4,Depth of layers\nd1/d2/d3/d4,Thickness Topsoil h1(m),...,Storage Fractured,TOTAL STORAGE,Logarithm of Topsoil,RESISTIVITY OF FRESH BASEMENT,Logarithm Fresh Basement,AQUIFER RES of Weathered. \n(Ohm-M),Logarithm Weathered,Hydraulic Conductivity (K),Transmissivity (T),Cost of Constructing Borehole
0,1,36.94,28.00,343.0,4.0,KH,128.3/213/52.7/137.7,1.1/4.9/23.4,1.1/6.1/29.5,1.1,...,0.0,30.018,2.108,137.7,2.139,52.7,1.722,9.5693,239.37241,301785.90
1,2,36.73,28.80,359.0,4.0,KH,518/583/252/752,0.8/1.6/22.7,0.8/2.5/25.2,0.8,...,0.0,29.120,2.714,752.0,2.876,252.0,2.401,2.2230,53.94388,271907.35
2,3,36.60,28.75,397.0,3.0,A,12.4/68/385.6,0.8/9.5,0.8/10.4,0.8,...,0.0,12.187,1.093,385.6,2.586,68.0,1.833,7.5443,76.61594,169069.55
3,4,36.84,29.58,342.0,3.0,H,309.8/174.1/3533.3,0.9/6.7,0.9/7.6,0.9,...,0.0,8.595,2.491,3533.3,3.548,174.1,2.241,3.1387,22.48047,150308.60
4,5,36.60,29.54,348.0,3.0,H,1303.7/292.7/940.5,2.5/62.2,2.5/64.7,2.5,...,0.0,79.790,3.115,940.5,2.973,292.7,2.466,1.9332,128.54409,547067.95


## Cleaning the data 
1. Remove the useless columns 
2. remove the features and labels 
3. Set the VES S/N as the main index class or whatever 

In [112]:
df.columns = [x.lower() for x in df.columns]

## Check the data and filter
check the list of rows 
then check their info 

filter and clean bad data

In [113]:
df.columns

Index(['ves s/n', 'easting (min)', 'northing (min)', 'elev(m)', 'no. of layer',
       'curve types', 'res. of layer\np1/p2/�pn-1',
       'thickness of layers h1/h2/h3/h4', 'depth of layers\nd1/d2/d3/d4',
       'thickness topsoil h1(m)', 'thickness laterite h2(m)',
       'thickness weathered h3 (m)', 'corrected thick. weathered (m)',
       'thickness fractured h4 (m)', 'corrcted thick. fractured (m)',
       'thick. overb.\nb1=h1+h2', 'total aquifer thick. b2=h3+h4',
       'corrected total aquifer thick', 'dtfb= b1 + b2', 'corrected dtb',
       'recom. dbh', 'p1', 'p2', 'p3', 'p4', 'p5', 'res. of topsoil',
       'res. of laterite', 'long. cond. (mhos) topsoil',
       'long. cond. (mhos) laterite', 'long. cond. (mhos) overburrden',
       'storage weathered', 'storage fractured', 'total storage',
       'logarithm of topsoil', 'resistivity of fresh basement',
       'logarithm fresh basement', 'aquifer res of weathered. \n(ohm-m)',
       'logarithm weathered', 'hydraulic conduc

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ves s/n                             255 non-null    object 
 1   easting (min)                       253 non-null    float64
 2   northing (min)                      253 non-null    float64
 3   elev(m)                             253 non-null    float64
 4   no. of layer                        253 non-null    float64
 5   curve types                         252 non-null    object 
 6   res. of layer
p1/p2/�pn-1           253 non-null    object 
 7   thickness of layers h1/h2/h3/h4     253 non-null    object 
 8   depth of layers
d1/d2/d3/d4         253 non-null    object 
 9   thickness topsoil h1(m)             255 non-null    float64
 10  thickness laterite h2(m)            255 non-null    float64
 11  thickness weathered h3 (m)          255 non-n

In [115]:
df_ = df.fillna(value={"p5": 2000, "p4" : 2000})

In [116]:
df_.dropna(inplace=True)

In [117]:
df_["ves s/n"] = [int(d) for d in df_["ves s/n"]]

In [118]:
df_.head()

,ves s/n,easting (min),northing (min),elev(m),no. of layer,curve types,res. of layer\np1/p2/�pn-1,thickness of layers h1/h2/h3/h4,depth of layers\nd1/d2/d3/d4,thickness topsoil h1(m),...,storage fractured,total storage,logarithm of topsoil,resistivity of fresh basement,logarithm fresh basement,aquifer res of weathered. \n(ohm-m),logarithm weathered,hydraulic conductivity (k),transmissivity (t),cost of constructing borehole
0,1,36.94,28.00,343.0,4.0,KH,128.3/213/52.7/137.7,1.1/4.9/23.4,1.1/6.1/29.5,1.1,...,0.0,30.018,2.108,137.7,2.139,52.7,1.722,9.5693,239.37241,301785.90
1,2,36.73,28.80,359.0,4.0,KH,518/583/252/752,0.8/1.6/22.7,0.8/2.5/25.2,0.8,...,0.0,29.120,2.714,752.0,2.876,252.0,2.401,2.2230,53.94388,271907.35
2,3,36.60,28.75,397.0,3.0,A,12.4/68/385.6,0.8/9.5,0.8/10.4,0.8,...,0.0,12.187,1.093,385.6,2.586,68.0,1.833,7.5443,76.61594,169069.55
3,4,36.84,29.58,342.0,3.0,H,309.8/174.1/3533.3,0.9/6.7,0.9/7.6,0.9,...,0.0,8.595,2.491,3533.3,3.548,174.1,2.241,3.1387,22.48047,150308.60
4,5,36.60,29.54,348.0,3.0,H,1303.7/292.7/940.5,2.5/62.2,2.5/64.7,2.5,...,0.0,79.790,3.115,940.5,2.973,292.7,2.466,1.9332,128.54409,547067.95


## Drop Repeated Data

This aspect will drop things like storage values and so on. 

In [119]:
df_.drop(df_.columns[[6,7,8,5,12,13,17,19,41,20]], axis=1, inplace=True)


In [131]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 252
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ves s/n                             252 non-null    int64  
 1   easting (min)                       252 non-null    float64
 2   northing (min)                      252 non-null    float64
 3   elev(m)                             252 non-null    float64
 4   no. of layer                        252 non-null    float64
 5   thickness topsoil h1(m)             252 non-null    float64
 6   thickness laterite h2(m)            252 non-null    float64
 7   thickness weathered h3 (m)          252 non-null    float64
 8   corrcted thick. fractured (m)       252 non-null    float64
 9   thick. overb.
b1=h1+h2              252 non-null    float64
 10  total aquifer thick. b2=h3+h4       252 non-null    float64
 11  dtfb= b1 + b2                       252 non-n

In [120]:
df_features = df_.drop(df_.columns[[0,22,23,24]], axis=1)
df_label = df_["total storage"]

## Split Training and testing data 

Finally split the data into the train and test datasets 


In [121]:
X_train, X_test, y_train, y_test = train_test_split(df_features, df_label)
X_train_scaled = scale(X_train)
X_test_scaled = scale(X_test)


# Training with linear regression classifier

In [127]:
clf = LinearRegression()

In [128]:
clf.fit(X_train, y_train)

LinearRegression()

In [130]:
accuracy = clf.score(X_test, y_test)
accuracy

0.9999999993703829